# Prerequisites

## For Server
- git clone https://github.com/most-medicalrecord -b develop
- cd most-medicalrecord
- make develop
- make sync
- make run

##For testing API with this notebook## 
- **Requests** library <http://docs.python-requests.org/en/latest/>

## For testing pyEHR API endpoint ##
- Download ova VirtualMachine from <http://156.148.18.70/files/pyEHR.ova>
- Run VirtualMachine (virtualbox or vmware importing vmdk virtual disk in new virtual machine)
- Configure VirtualMachine networking with NAT or BRIDGE
- Wait for services start (up to 5 minutes)
- Test dbservice status calling <http://virtualmachine:8080/check/status/dbservice>
- Configure PYEHR_DB_SERVICE_IP on django settings.py
- stop SERVER and restart with **make run**

In [1]:
#Get Oauth2 Access Token specifying existing taskgroup_uuid

import requests
import json

server_port = 9000

arguments = {
             'client_id': '8c96bf8cea26fa555fa8',
             'client_secret': '4fd1f508b7b03fba6509da4c193157d7a2b20838',
             'grant_type': 'password',
             'username': 'admin',
             'password': 'admin',
             'taskgroup': '5dw2x3jfkftxue5a5izw6yiplbbn4dlo'
             }
r = requests.post(
                  'http://127.0.0.1:{server_port}/oauth2/access_token/'.format(server_port=server_port), 
                  data=arguments
                  )
print r.text

access_token = json.loads(r.text)['access_token']

{"access_token": "e41edf9e651311575c119b07391e6a62011155c2", "token_type": "Bearer", "expires_in": 2591999, "scope": "read"}


In [2]:
#Test authenticated url

payload = {'access_token': access_token}
r = requests.get(
                    'http://127.0.0.1:{server_port}/medicalrecords/ping/auth/'.format(server_port=server_port), 
                    params=payload
                  )
print r.text

authenticated pong


In [19]:
#Create new patient record with existing demographics and ehr uuid

payload = {'access_token': access_token, 'ehr_uuid' : 'PATIENT_00'}

r = requests.post(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text

{"patient": {"demographic_uuid": "w5hsc5z2hzw7q2igevsjldxjnjkrc27q", "ehr_uuid": "PATIENT_00", "uuid": "iomrspncvrufkhx6mev5pjhaaclsluxl"}, "success": true}


In [20]:
#Retrieve all patients for taskgroup
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/'.format(server_port=server_port), 
    params=payload
)
print r.text
data = json.loads(r.text)
patient_uuid = data['patients'][0]['uuid']

{"patients": [{"demographic_uuid": "w5hsc5z2hzw7q2igevsjldxjnjkrc27q", "ehr_uuid": "PATIENT_00", "uuid": "iomrspncvrufkhx6mev5pjhaaclsluxl"}], "success": true}


In [21]:
#Retrieve patient by uuid
payload = {'access_token': access_token}

r = requests.get(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload
)
print r.text

{"patient": {"demographic_uuid": "w5hsc5z2hzw7q2igevsjldxjnjkrc27q", "ehr_uuid": "PATIENT_00", "uuid": "iomrspncvrufkhx6mev5pjhaaclsluxl"}, "success": true}


In [16]:
#Update patient data by uuid
payload = {'access_token': access_token, 'ehr_uuid' : 'TEST_EHR_ID3'}

r = requests.put(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload, data=data
)
print r.text

{"patient": {"demographic_uuid": "4l3xgugymbdfcvuv26h6fnklie56b53y", "ehr_uuid": "TEST_EHR_ID3", "uuid": "6yiz3nzbd3dgnonguedmuh3kibbdkzqy"}, "success": true}


In [17]:
#Delete patient by uuid
payload = {'access_token': access_token}

r = requests.delete(
    'http://127.0.0.1:{server_port}/medicalrecords/patients/{patient_uuid}/'.format(server_port=server_port, patient_uuid=patient_uuid), 
    params=payload, data=data
)
print r.text

{"patient": {"demographic_uuid": "4l3xgugymbdfcvuv26h6fnklie56b53y", "ehr_uuid": "TEST_EHR_ID3", "uuid": "6yiz3nzbd3dgnonguedmuh3kibbdkzqy"}, "success": true}
